# Orthogonality Data Preparation

**Equivalent to:** `data7_orthogonality.do`

**Purpose:** Creating orthogonality table data from the main customer dataset

This notebook recreates the data processing pipeline that:
1. Loads the main customer dataset (`all.dta`)
2. Filters to pre-campaign period (September 2011 - August 2012) 
3. Cleans and processes variables for balance checks
4. Creates customer-level aggregated variables
5. Saves as `orthogonality.parquet` for analysis

**Original Stata Code Logic:**
- Keeps only pre-campaign period data
- Cleans numeric variables (removes "(null)", converts to numeric)
- Creates customer-level means for key variables
- Collapses to one row per customer


In [ ]:
# Setup
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root / 'config'))

import config
import pyreadstat
import pandas as pd
import numpy as np
from datetime import datetime

print("Orthogonality Data Preparation")
print("=" * 40)
print(f"Equivalent to: data7_orthogonality.do")
print(f"Input: {config.DATASETS['all']}")
print(f"Output: {config.ANALYSIS_DATA_DIR / 'orthogonality.parquet'}")


Orthogonality Data Preparation
Equivalent to: data7_orthogonality.do
Input: /Users/zenofficial/Documents/statistics/pcs/turkey_python_analysis/data/bankdata/clean/all.dta
Output: /Users/zenofficial/Documents/statistics/pcs/turkey_python_analysis/data/analysis/orthogonality.parquet


In [ ]:
# Load main dataset directly from Stata file
print("Loading main customer dataset...")

df, meta = pyreadstat.read_dta(str(config.DATASETS['all']))
print(f"✓ Loaded from Stata file: {df.shape}")

print(f"Date range in data: {df['date'].min()} to {df['date'].max()}")
print(f"Unique customers: {df['id'].nunique()}")
print(f"Sample columns: {list(df.columns[:10])}")


Loading main customer dataset...
✓ Loaded from parquet: (1836354, 42)
Date range in data: 619.0 to 636.0
Unique customers: 108000
Sample columns: ['id', 'date', 'before', 'TreatVars', 'treatment', 'desc', 'StratVars', 'cinsiyet', 'kametili', 'istanbul']


In [ ]:
# Clean numeric variables first (equivalent to Stata foreach loop)
print("\nCleaning numeric variables...")

def clean_numeric_variables(df, variables):
    """Clean numeric variables (replaces "(null)" with NaN, converts to numeric)"""
    df_clean = df.copy()
    for var in variables:
        if var in df_clean.columns:
            df_clean[var] = df_clean[var].replace("(null)", np.nan)
            df_clean[var] = pd.to_numeric(df_clean[var], errors='coerce')
            df_clean[var] = df_clean[var].fillna(0)
    return df_clean

numeric_vars = ['debt_tot', 'debt_term', 'transnum', 'instrucnum', 'cardsnum']
df_clean = clean_numeric_variables(df_filtered, numeric_vars)

# Handle other variables
if 'creditcard' in df_clean.columns:
    df_clean['creditcard'] = df_clean['creditcard'].fillna(0)
if 'credit' in df_clean.columns:    
    df_clean['credit'] = df_clean['credit'].fillna(0)
if 'faamount' in df_clean.columns:
    df_clean['faamount'] = df_clean['faamount'].fillna(0)

print("✓ Numeric variables cleaned")

# Create customer-level aggregated variables (equivalent to Stata egen commands)
print("\nCreating customer-level aggregated variables...")

# Calculate customer-level means for key variables
aggregation_vars = {
    'transactions': 'transnum',
    'assets': 'a_total', 
    'deposits': 'a_deposit',
    'paymentmean': 'payment',
    'debt': 'debt_tot'
}

df_agg = df_clean.copy()

for new_var, source_var in aggregation_vars.items():
    if source_var in df_agg.columns:
        df_agg[new_var] = df_agg.groupby('id')[source_var].transform('mean')
        print(f"✓ Created {new_var} from {source_var}")

# Handle pastuse variable (equivalent to Stata pastuse creation)
if 'faamount' in df_agg.columns:
    df_agg['pastuse'] = df_agg.groupby('id')['faamount'].transform('mean')
    df_agg['pastuse'] = (df_agg['pastuse'] != 0).astype(int)
    print("✓ Created pastuse variable")

# Handle autobillpay variable 
if 'var3' in df_agg.columns:
    df_agg['autobillpay'] = df_agg['var3'].fillna(0)
    df_agg['autobillpay'] = df_agg.groupby('id')['autobillpay'].transform('mean')
    df_agg['autobillpay'] = (df_agg['autobillpay'] != 0).astype(int)
    print("✓ Created autobillpay variable")

print(f"Data shape after aggregation: {df_agg.shape}")



Cleaning numeric variables...
✓ Numeric variables cleaned

Creating customer-level aggregated variables...
✓ Created transactions from transnum
✓ Created assets from a_total
✓ Created deposits from a_deposit
✓ Created paymentmean from payment
✓ Created debt from debt_tot
✓ Created pastuse variable
✓ Created autobillpay variable
Data shape after aggregation: (1296000, 49)


In [5]:
# Collapse to one row per customer (equivalent to Stata collapse command)
print("\nCollapsing to customer level...")

# Define variables to keep and their aggregation methods
keep_vars = ['id', 'treatment', 'phase1treat', 'cinsiyet', 'city', 'medenihal', 
             'falimit', 'transactions', 'assets', 'deposits', 'paymentmean', 
             'debt', 'pastuse', 'autobillpay', 'acctbalance']

# Variables that use first non-missing value (firstnm in Stata)
first_vars = ['treatment', 'phase1treat', 'cinsiyet', 'city', 'medenihal', 
              'falimit', 'transactions', 'assets', 'deposits', 'paymentmean', 
              'debt', 'pastuse', 'autobillpay']

# Variables that use last non-missing value (lastnm in Stata)  
last_vars = ['creditcard', 'credit']

# Variables that use sum
sum_vars = ['faamount']

# Build aggregation dictionary
agg_dict = {}
for var in first_vars:
    if var in df_agg.columns:
        agg_dict[var] = 'first'

for var in last_vars:
    if var in df_agg.columns:
        agg_dict[var] = 'last'
        
for var in sum_vars:
    if var in df_agg.columns:
        agg_dict[var] = 'sum'

# Perform the collapse
df_collapsed = df_agg.groupby('id').agg(agg_dict).reset_index()

print(f"✓ Collapsed to {len(df_collapsed):,} customers")
print(f"Final dataset shape: {df_collapsed.shape}")
print(f"Variables: {list(df_collapsed.columns)}")



Collapsing to customer level...
✓ Collapsed to 108,000 customers
Final dataset shape: (108000, 17)
Variables: ['id', 'treatment', 'phase1treat', 'cinsiyet', 'city', 'medenihal', 'falimit', 'transactions', 'assets', 'deposits', 'paymentmean', 'debt', 'pastuse', 'autobillpay', 'creditcard', 'credit', 'faamount']


In [6]:
# Add variable labels and save final dataset
print("\nFinalizing dataset...")

# Add variable labels (equivalent to Stata label var commands)
var_labels = {
    'treatment': 'Treatment',
    'phase1treat': 'Treatment in First Phase Randomization',
    'cinsiyet': 'Gender',
    'city': 'City',
    'medenihal': 'Marital Status',
    'falimit': 'Overdraft Limit',
    'transactions': 'Average Monthly Transactions',
    'assets': 'Average Monthly Assets',
    'deposits': 'Average Monthly Deposits', 
    'debt': 'Average Monthly Debt',
    'pastuse': 'Past Overdraft Use (Binary)',
    'autobillpay': 'Auto Bill Pay (Binary)'
}

# Show summary statistics
print("Summary of key variables:")
summary_vars = ['treatment', 'transactions', 'assets', 'deposits', 'debt', 'pastuse']
existing_summary_vars = [var for var in summary_vars if var in df_collapsed.columns]

if existing_summary_vars:
    summary_stats = df_collapsed[existing_summary_vars].describe()
    print(summary_stats.round(3))

# Check treatment balance
if 'treatment' in df_collapsed.columns:
    treatment_counts = df_collapsed['treatment'].value_counts()
    print(f"\nTreatment assignment counts:")
    print(treatment_counts)

# Save the final dataset
output_path = config.ANALYSIS_DATA_DIR / 'orthogonality.parquet'
df_collapsed.to_parquet(output_path, index=False)
print(f"\n✓ Saved orthogonality dataset to: {output_path}")
print(f"✓ Dataset ready for Table 1 analysis")

# Also save as CSV for easy viewing
csv_path = config.ANALYSIS_DATA_DIR / 'orthogonality.csv'
df_collapsed.to_csv(csv_path, index=False)
print(f"✓ Also saved as CSV: {csv_path}")



Finalizing dataset...
Summary of key variables:
        treatment  transactions      assets    deposits        debt  \
count  108000.000    108000.000  107773.000  107773.000  108000.000   
mean       18.487         1.636     638.896     377.300     283.393   
std        10.390         4.213    2002.738    1032.487     715.635   
min         1.000         0.000       0.000       0.000   -1733.083   
25%         9.000         0.000      42.911      36.912       0.000   
50%        18.000         0.000     147.350     118.202       0.000   
75%        27.000         1.083     531.393     321.458     222.250   
max        36.000        88.583  120975.639   54395.918   15057.083   

          pastuse  
count  108000.000  
mean        0.304  
std         0.460  
min         0.000  
25%         0.000  
50%         0.000  
75%         1.000  
max         1.000  

Treatment assignment counts:
treatment
9.0     3017
3.0     3014
2.0     3013
12.0    3011
15.0    3010
6.0     3009
27.0    3008


In [7]:
# Show summary of key variables after cleaning
if 'debt_tot' in df_clean.columns:
    print(f"debt_tot: min={df_clean['debt_tot'].min()}, max={df_clean['debt_tot'].max()}, null_count={df_clean['debt_tot'].isnull().sum()}")

print("\n✓ Data cleaning and aggregation completed successfully")


debt_tot: min=-10271.0, max=42099.0, null_count=0

✓ Data cleaning and aggregation completed successfully
